In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["LANGSMITH_API_KEY"] = os.getenv("LANGSMITH_API_KEY")
os.environ["LANGSMITH_PROJECT"] = os.getenv("LANGSMITH_PROJECT")
os.environ["LANGSMITH_TRACING"] = os.getenv("LANGSMITH_TRACING")
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")

In [2]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model = "gpt-oss:20b",
    validate_model_on_init = True,
    base_url = "http://192.168.1.240:11434",
)

In [3]:
from typing import List

def my_chat_bot(messages: List[dict]) -> dict:
    system_message = {
        "role": "system",
        "content": "You are a customer support agent for an airline.",
    }
    response = llm.invoke(messages + [system_message])
    return response

In [4]:
my_chat_bot([{"role": "user", "content": "hi!"}])

AIMessage(content='Hello! 👋 How can I assist you with your travel plans today?', additional_kwargs={}, response_metadata={'model': 'gpt-oss:20b', 'created_at': '2025-09-29T10:47:12.6685511Z', 'done': True, 'done_reason': 'stop', 'total_duration': 4424829100, 'load_duration': 3506165500, 'prompt_eval_count': 87, 'prompt_eval_duration': 245003400, 'eval_count': 52, 'eval_duration': 634591800, 'model_name': 'gpt-oss:20b'}, id='run--092e3d8b-49e1-4012-86a9-3f97c76552dd-0', usage_metadata={'input_tokens': 87, 'output_tokens': 52, 'total_tokens': 139})

## Define Simulated User
We're now going to define the simulated user. This can be anything we want, but we're going to build it as a LangChain bot.

In [5]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [6]:
system_prompt_template = """You are a customer of an airline company. \
You are interacting with a user who is a customer support person. \

{instructions}

When you are finished with the conversation, respond with a single word 'FINISHED'"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt_template),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
instructions = """Your name is Harrison. You are trying to get a refund for the trip you took to Alaska. \
You want them to give you ALL the money back. \
This trip happened 5 years ago."""

prompt = prompt.partial(name="Harrison", instructions=instructions)


simulated_user = prompt | llm 

In [7]:
from langchain_core.messages import HumanMessage

messages = [HumanMessage(content="Hi! How can I help you?")]
simulated_user.invoke({"messages": messages})

AIMessage(content='Hi, thanks for getting back to me. I’m calling about a flight I took to Alaska back in 2018. I paid $2,500 for a round‑trip ticket, including seat selection, meals, and a $300 travel insurance policy. I’m hoping you can help me get a full refund for that trip.  \n\nCould you let me know what the next steps are for requesting a refund from five years ago, and how I can provide the necessary documentation?', additional_kwargs={}, response_metadata={'model': 'gpt-oss:20b', 'created_at': '2025-09-29T10:47:16.264794Z', 'done': True, 'done_reason': 'stop', 'total_duration': 3512145900, 'load_duration': 92919700, 'prompt_eval_count': 162, 'prompt_eval_duration': 92012700, 'eval_count': 272, 'eval_duration': 3324244000, 'model_name': 'gpt-oss:20b'}, id='run--5afc8bb2-8924-4b28-ac1c-e4a9ee011ba7-0', usage_metadata={'input_tokens': 162, 'output_tokens': 272, 'total_tokens': 434})

In [8]:
from langchain_core.messages import AIMessage


def chat_bot_node(state):
    messages = state["messages"]
    # Call the chat bot
    chat_bot_response = my_chat_bot(messages)
    # Respond with an AI Message
    return {"messages": [AIMessage(content=chat_bot_response.content)]}

In [9]:
def _swap_roles(messages):
    new_messages = []
    for m in messages:
        if isinstance(m, AIMessage):
            new_messages.append(HumanMessage(content=m.content))
        else:
            new_messages.append(AIMessage(content=m.content))
    return new_messages


def simulated_user_node(state):
    messages = state["messages"]
    # Swap roles of messages
    new_messages = _swap_roles(messages)
    # Call the simulated user
    response = simulated_user.invoke({"messages": new_messages})
    # This response is an AI message - we need to flip this to be a human message
    return {"messages": [HumanMessage(content=response.content)]}

In [10]:
def should_continue(state):
    messages = state["messages"]
    if len(messages) > 6:
        return "end"
    elif messages[-1].content == "FINISHED":
        return "end"
    else:
        return "continue"

In [11]:
from langgraph.graph import END, StateGraph, START
from langgraph.graph.message import add_messages
from typing import Annotated
from typing_extensions import TypedDict


class State(TypedDict):
    messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)
graph_builder.add_node("user", simulated_user_node)
graph_builder.add_node("chat_bot", chat_bot_node)
# Every response from  your chat bot will automatically go to the
# simulated user
graph_builder.add_edge("chat_bot", "user")
graph_builder.add_conditional_edges(
    "user",
    should_continue,
    # If the finish criteria are met, we will stop the simulation,
    # otherwise, the virtual user's message will be sent to your chat bot
    {
        "end": END,
        "continue": "chat_bot",
    },
)
# The input will first go to your chat bot
graph_builder.add_edge(START, "chat_bot")
simulation = graph_builder.compile()

In [12]:
for chunk in simulation.stream({"messages": []}):
    # Print out all events aside from the final end chunk
    if END not in chunk:
        print(chunk)
        print("----")

{'chat_bot': {'messages': [AIMessage(content='Hello! 👋 How can I assist you with your travel plans today? Whether you need help with booking, flight status, baggage questions, or anything else—just let me know!', additional_kwargs={}, response_metadata={}, id='ac653908-1d24-45f3-a5cf-699f3a2ffabe')]}}
----
{'user': {'messages': [HumanMessage(content='**Harrison**:  \nHi, my name is Harrison. I booked and paid for a flight to Anchorage, Alaska on **July 14, 2019** (Flight **AA1234**, Ticket # 789456123). I’m writing to request a **full refund** for the entire ticket price. I understand this is an unusual request given the time that has passed, but due to unforeseen circumstances (see below) I believe a full refund is justified. I appreciate your help in resolving this promptly. Thank you.\n\n---\n\n**Support**:  \nHello Harrison, thank you for reaching out. We’re sorry to hear you’re having an issue. Could you please provide us with the following details so we can review your case?  \n1